In [1]:
import numpy as np
import os
import preprocessing_functions as pf
import cv2 as cv
import csv
import pandas as pd

In [2]:
def load_img(meta_path='/solar/sat/art-rrr/ART_G010x010RR_C010x010/meta',
             last_img_name='ART_2020020_111017.FR' ,
             mk_folder_path='/solar/sat/art-rrr/ART_G010x010RR_C010x010/C02-MK/2020',
             img_folder_path='/solar/sat/art-rrr/ART_G010x010RR_C010x010/C02-FR/2020'):
 
    lats, lons = pf.read_meta(meta_path)
    dtime = pf.get_dtime(last_img_name)
    cosangs, cos_mask = pf.get_cosangs(dtime, lats, lons)
    img_mask = pf.load_mask(last_img_name, mk_folder_path, lats.size, lons.size)
    img = pf.load_img(last_img_name, img_folder_path, lats.size, lons.size)
    rimg = cv.inpaint(img, img_mask, 3, cv.INPAINT_NS)
    rp_image = pf.normalize(rimg, cosangs, 0.15)
    return rp_image  

In [7]:
#Muy lento, buscar forma de optimizar
#Demora 35 seg para 100 imagenes, para 51219 se espera 5 hs aprox

#Load first img from folder

fr_folder_path='/solar/sat/art-rrr/ART_G010x010RR_C010x010/C02-FR/2020'
img1_filename = sorted(os.listdir(fr_folder_path))[0]
img1 = load_img(last_img_name=img1_filename)


#Iterate from second img to last./,;'[//,/pl
fieldnames = ['Img1', 'Img2', 'rmse']

with open(   'outliers2.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for filename in sorted(os.listdir(fr_folder_path))[1:]:
        img2_filename = filename
        img2 = load_img(last_img_name=img2_filename)
        rmse = np.sqrt(np.mean((img1 - img2)**2))
        
        row_dict = {}
        row_dict['Img1'] = img1_filename
        row_dict['Img2'] = img2_filename
        row_dict['rmse'] = rmse

        writer.writerow(row_dict)
        
        img1 = img2
        img1_filename = img2_filename
    